In [2]:
!pip install pyngrok
!pip install flask-ngrok
!pip install flask_cors

In [3]:
!pip install emoji
!pip install contractions
!pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 9.7 MB/s eta 0:00:00


In [4]:
import re
import nltk
import emoji
import contractions
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Supprimer les caractères répétés
def reduire_caracteres_repetes(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

# Supprimer les emojis
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

# Supprimer les caractères non ASCII (symboles)
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

# Fonction principale de nettoyage
def nettoyer_texte(tweet):
    tweet = tweet.lower()  # minuscules
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)  # URL
    tweet = re.sub(r"@\w+", "", tweet)  # mentions
    tweet = re.sub(r"\d+", "", tweet)  # nombres
    tweet = re.sub(r"[^\w\s]", "", tweet)  # ponctuation
    tokens = tweet.split()  # découper en mots
    tokens = [w for w in tokens if w not in stop_words]  # stopwords
    tokens = [stemmer.stem(w) for w in tokens]  # stemming
    return " ".join(tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from typing import Text
from flask import Flask, request, jsonify
from pyngrok import ngrok
import numpy as np
from keras.models import load_model
import pickle
from keras.preprocessing.sequence import pad_sequences
from flask_cors import CORS


ngrok.set_auth_token("2xvlEQpbExoxLpciQvbU53Zr99r_3XdjiPteDruJrQr1zC2gP")
public_url = ngrok.connect(5000)
print("ngrok tunnel URL:", public_url)

model = load_model('/content/drive/MyDrive/cyberbullying_model.keras')
with open('/content/drive/MyDrive/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open('/content/drive/MyDrive/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

def preprocess(text):
    print("texte avant nettoyage  ",text)
    text =nettoyer_texte(text)
    print("texte nettoyer est :",text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=100)
    return padded

app = Flask(__name__)
CORS(app, origins=["http://localhost:3000"])
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data['text']
    processed = preprocess(text)
    pred = model.predict(processed)

    class_id = np.argmax(pred,axis=1)
    class_label = label_encoder.inverse_transform(class_id)[0]
    return jsonify({'prediction': class_label})
app.run()


ngrok tunnel URL: NgrokTunnel: "https://58ad-34-125-168-22.ngrok-free.app" -> "http://localhost:5000"


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
